In [1]:
!git clone https://github.com/graykode/gpt-2-Pytorch
%cd gpt-2-Pytorch
!curl --output gpt2-pytorch_model.bin https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin
!pip install -r requirements.txt


fatal: destination path 'gpt-2-Pytorch' already exists and is not an empty directory.
/Users/manyadua/Downloads/ryan-sus/capstone/MadLib/gpt-2-Pytorch
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  522M  100  522M    0     0  2712k      0  0:03:18  0:03:17  0:00:01  9.9M 0     0      0      0 --:--:-- --:--:-- --:--:--     0  0     0  2416k      0  0:03:41  0:00:32  0:03:09 2388k      0  0:03:40  0:00:33  0:03:07 2464k  0  2596k      0  0:03:26  0:00:43  0:02:43 3606k0     0  2634k      0  0:03:23  0:00:44  0:02:39 3831k:03:17  0:00:46  0:02:31 4032k:48  0:02:26 4048k2794k      0  0:03:11  0:01:21  0:01:50 2294k    0     0  2779k      0  0:03:12  0:01:23  0:01:49 2198k  0:03:13  0:01:26  0:01:47 2391k 0  2772k      0  0:03:13  0:01:27  0:01:46 2546k3:18  0:01:54  0:01:24 2601k   0  2693k      0  0:03:18  0:01:55  0:01:23 2604k9k      0  0:03:20  0:01:59  0:01:21 2104k    

In [4]:
# @param: string
# @return: parts of speech in a list 

def get_nouns(sentence):
    
    print(sentence)

#Level 1 
# Harry Potter - nouns -> story 
# Marvel - nouns -> story 

#Level 2 
# they give 2-5 words -> story 

#Level 3 
# Madlib 

#Crazy level -> use our model to generate random sentences 



In [ ]:
'''
    code by TaeHwan Jung(@graykode)
    Original Paper and repository here : https://github.com/openai/gpt-2
    GPT2 Pytorch Model : https://github.com/huggingface/pytorch-pretrained-BERT
'''
import os
import sys
import torch
import random
import argparse
import numpy as np
from GPT2.model import (GPT2LMHeadModel)
from GPT2.utils import load_weight
from GPT2.config import GPT2Config
from GPT2.sample import sample_sequence
from GPT2.encoder import get_encoder

def text_generator(state_dict):
    parser = argparse.ArgumentParser()
    parser.add_argument("--text", type=str, required=True)
    parser.add_argument("--quiet", type=bool, default=False)
    parser.add_argument("--nsamples", type=int, default=1)
    parser.add_argument('--unconditional', action='store_true', help='If true, unconditional generation.')
    parser.add_argument("--batch_size", type=int, default=-1)
    parser.add_argument("--length", type=int, default=-1)
    parser.add_argument("--temperature", type=float, default=0.7)
    parser.add_argument("--top_k", type=int, default=40)
    args = parser.parse_args()

    if args.quiet is False:
        print(args)

    if args.batch_size == -1:
        args.batch_size = 1
    assert args.nsamples % args.batch_size == 0

    seed = random.randint(0, 2147483647)
    np.random.seed(seed)
    torch.random.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load Model
    enc = get_encoder()
    config = GPT2Config()
    model = GPT2LMHeadModel(config)
    model = load_weight(model, state_dict)
    model.to(device)
    model.eval()

    if args.length == -1:
        args.length = config.n_ctx // 2
    elif args.length > config.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % config.n_ctx)

    print(args.text)
    context_tokens = enc.encode(args.text)

    generated = 0
    for _ in range(args.nsamples // args.batch_size):
        out = sample_sequence(
            model=model, length=args.length,
            context=context_tokens  if not  args.unconditional else None,
            start_token=enc.encoder['<|endoftext|>'] if args.unconditional else None,
            batch_size=args.batch_size,
            temperature=args.temperature, top_k=args.top_k, device=device
        )
        out = out[:, len(context_tokens):].tolist()
        
        for i in range(args.batch_size):
            generated += 1
            text = enc.decode(out[i])
            if args.quiet is False:
                print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
            print(text)

if __name__ == '__main__':
    if os.path.exists('gpt2-pytorch_model.bin'):
        state_dict = torch.load('gpt2-pytorch_model.bin', map_location='cpu' if not torch.cuda.is_available() else None)
        text_generator(state_dict)
    else:
        print('Please download gpt2-pytorch_model.bin')
        sys.exit()


In [3]:
#Once w00hen I was six years old I saw a magnificent picture in a book, called True Stories from Nature, about the primeval forest.
text = !python main.py --text "Marvel" 

IPython.utils.text.SList

In [11]:
print(type(text))
for i in text:
    print(i)
    
# print(text.find('=='))

<class 'IPython.utils.text.SList'>
Namespace(text='Marvel', quiet=False, nsamples=1, unconditional=False, batch_size=-1, length=-1, temperature=0.7, top_k=40)
Marvel

100%|██████████| 512/512 [01:40<00:00,  5.11it/s]
======================================== SAMPLE 1 ========================================
, and I do have a few ideas for how to get more players involved in the game, but I'm pretty sure none of them will be a thing.

Are you working on a third game?

In the next year or two, we're going to do something different and put a lot of people on a game that is very close to the original game. We're all going to do what we're doing, but we'll also be giving the game away as a free download and we'll be giving away games of our own, so if you give away a game in the game store, you'll get something free. If you give away a game in the game store, you'll get a free game. We're also going to give away some DLC that's coming out.

I've tried to do a lot of things with my time, but 

In [ ]:
# text -> string of paragraphs

list_words = []

for i in range(1,6):
    list_words.append(input("Please enter noun " + str(i) + ": "))
    

In [6]:
from cogworks_data.language import get_data_path

pj = "This is the sentence I am trying to find things for "

#get_data_path("pjolympians.txt")

pj = pj.lower()  
pj.split()

lmpj1 = train_lm(pj, 11)
new_text = generate_text(lmpj1, 11,200)
print(new_text)

# noun = input("enter a noun!")

# print(noun)




ModuleNotFoundError: No module named 'cogworks_data'